In [1]:
import psycopg2
import requests

In [2]:
from jsonpath_ng import jsonpath,parse

In [3]:
session=requests.session()

In [4]:
dataJsonURL="https://api.covid19india.org/v4/data.json"
stateWiseJsonURL="https://api.covid19india.org/state_district_wise.json"

In [5]:
try:
    connection = psycopg2.connect(user = "postgres",
                                  password = "telelab",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "covmap")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except:
    print("ERROR")

{'user': 'postgres', 'dbname': 'covmap', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 12.4, compiled by Visual C++ build 1914, 64-bit',) 



In [28]:
dataStateWise=session.get(stateWiseJsonURL)
dataV4=session.get(dataJsonURL)

In [10]:
import pandas as pd

In [38]:
query='SELECT engtype_1,state,statecode from public."IND_STATES2"'

In [39]:
outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)

In [40]:
with open('statesList', 'w') as f:
    cursor.copy_expert(outputquery, f)


In [19]:
districtList=pd.read_csv('districtList.csv')

In [20]:
districtList

,engtype_2,district,state,statecode
0,District,east_godavari,andhra_pradesh,AD
1,District,nicobar_islands,andaman_and_nicobar,AN
2,District,guntur,andhra_pradesh,AD
3,District,karimganj,assam,AS
4,District,krishna,andhra_pradesh,AD
...,...,...,...,...
686,District,amethi,uttar_pradesh,UP
687,District,amroha,uttar_pradesh,UP
688,District,mau,uttar_pradesh,UP
689,District,sambhal,uttar_pradesh,UP


In [21]:
apiList=pd.read_csv('apilist.csv')

In [22]:
apiList

,covid19Name,apiName
0,Adilabad,adilabad
1,Agar Malwa,agar_malwa
2,Agra,agra
3,Ahmedabad,ahmadabad
4,Ahmednagar,ahmadnagar
...,...,...
724,Yadgir,yadgir
725,Yamunanagar,yamunanagar
726,Yanam,yanam
727,Yavatmal,yavatmal


In [24]:
df_merge_difkey = pd.merge(districtList, apiList, left_on='district', right_on='apiName')

In [25]:
df_merge_difkey

,engtype_2,district,state,statecode,covid19Name,apiName
0,District,east_godavari,andhra_pradesh,AD,East Godavari,east_godavari
1,District,nicobar_islands,andaman_and_nicobar,AN,Nicobars,nicobar_islands
2,District,guntur,andhra_pradesh,AD,Guntur,guntur
3,District,karimganj,assam,AS,Karimganj,karimganj
4,District,krishna,andhra_pradesh,AD,Krishna,krishna
...,...,...,...,...,...,...
729,District,y_s_r,andhra_pradesh,AD,Y.S.R. Kadapa,y_s_r
730,District,amethi,uttar_pradesh,UP,Amethi,amethi
731,District,amroha,uttar_pradesh,UP,Amroha,amroha
732,District,mau,uttar_pradesh,UP,Mau,mau


In [26]:
df_merge_difkey.to_csv ('finalList.csv', index = None)

In [37]:
# for s in dataStateWise.json().keys():
    # print(s+',')

In [ ]:
outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)

In [44]:
finalList=pd.read_csv('finalList.csv')

In [49]:
def replac(row):
    if row.type=="District":
        return "d"
    if row.type=="State":
        return "s"
    return "u"

In [50]:
finalList["type"]=finalList.apply(replac,axis=1)

In [52]:
finalList.to_csv('finalList.csv',index=None)